In [1]:
import os

import pandas as pd

In [ ]:
#construccion ruta datos Bronce donde estan datos Bronce
bronce_path = os.path.join(
    os.curdir, "Data_Lake", "Bronce"
)

#construccion ruta datos Silver
silver_path = os.path.join(
    os.curdir, "Data_Lake", "Silver"
)

# Nombre del archivo parquet de entrada (donde datos originales)
bronce_file_name = "erp_bronce.parquet"

# Nombre del archivo parquet de salida (donde guardo datos procesados-limpios)
silver_file_name = "erp_silver.parquet"

# Ruta completa del archivo Bronce y Silver
bronce_file_path = os.path.join(
    bronce_path, bronce_file_name
)
silver_file_path = os.path.join(
    silver_path, silver_file_name
)

# lectura datos bronce (primeras filas)
bronce_erp_df = pd.read_parquet(bronce_file_path)
bronce_erp_df.head()


# Compruebo si la columna de id contiene nulos (si aperecieron)
bronce_erp_df[bronce_erp_df['RAZONSOCIAL'].isnull() == True]

#bronce_erp_df.info ()


# Borramos filas donde la columna es nula (ok) (eliminacion de 2 filas)
bronce_erp_df = bronce_erp_df.dropna(subset=["RAZONSOCIAL"])

bronce_erp_df[bronce_erp_df['RAZONSOCIAL'].isnull() == True]



#Aplico otro filtro para eliminar las filas si valores nulos en conjunto de columnas (identificacion empresas)
#puedo usar "any" para eliminar fila si algun valor nulo en alguna de las columnas
#bronce_erp_df es el df
resto_identificacion = ["SITIOWEB", "DIRECCION", "EMAIL", "TELEFONO"]
bronce_erp_df = bronce_erp_df.dropna(subset=resto_identificacion, how="all")



#ELIMINACION COLUMNAS INNECESARIAS

bronce_erp_df = bronce_erp_df.drop(columns=['TELEFONO2', 'FAX'])

#bronce_erp_df.info()

# Defino como voy a tratar los nulos en el resto de columnas (no trato empleo total ni rango facturacion)
#apunte: df.fillna(valor_o_diccionario), aqui vamos con el diccionario en la variable rellenar_valores
rellenar_valores = {
    "NOMBRECOMERCIAL": "No especificado",
    "DIRECCION": "No especificado",
    "SITIOWEB": "No disponible",
    "EMAIL": "No disponible",
    "TELEFONO": "No disponible",
    "PRODUCTOS": "No especificado",
    "IMPORTA": "No",
    "EXPORTA": "No"
}

# Aplico la limpieza
bronce_erp_clean_df = bronce_erp_df.fillna(rellenar_valores)

#bronce_erp_clean_df.info ()

#VALORES ATIPICOS
#RAZON SOCIAL vemos si hay datos repetidos
bronce_erp_clean_df['RAZONSOCIAL'].value_counts(dropna=False)

#bronce_erp_clean_df['RAZONSOCIAL'].count()
#quitamos filas duplicados RAZON SOCIAL
bronce_erp_clean_df = bronce_erp_clean_df.drop_duplicates('RAZONSOCIAL')
bronce_erp_clean_df['RAZONSOCIAL'].value_counts()


# NOMBRECOMERCIAL varios nombres duplicados, 2 empresas duplicadas (ALSA, COVEMYM, HICASA, TKE) y otros 3 completo nombres: 1. COMPLETO NOMBRES 2. ELIMINO FILAS DUPLICADAS
pd.set_option('display.max_rows', None)
bronce_erp_clean_df['NOMBRECOMERCIAL'].value_counts(dropna=False)


bronce_erp_clean_df.loc[bronce_erp_clean_df["RAZONSOCIAL"] == "APLICACIONES DE SUPERFICIES DE ASTURIAS, S.L.", "NOMBRECOMERCIAL"] = "ASA GIJON"
bronce_erp_clean_df.loc[bronce_erp_clean_df["RAZONSOCIAL"] == "ASTURIANA DE CALCITAS, S.L.", "NOMBRECOMERCIAL"] = "ASTURIANA DE CALCITAS"
bronce_erp_clean_df.loc[bronce_erp_clean_df["RAZONSOCIAL"] == "AESVAL LOGICA DE VALORACIONES, S.A.","NOMBRECOMERCIAL" ] = "AESVAL"


bronce_erp_clean_df = bronce_erp_clean_df.drop_duplicates('NOMBRECOMERCIAL')
bronce_erp_clean_df['NOMBRECOMERCIAL'].value_counts(dropna=False)


#DIRECCION ( 8 direcciones duplicadas, aceptable)
bronce_erp_clean_df['DIRECCION'].value_counts(dropna=False)

#WEB (12 empresas se duplican, aceptable, pueden pertenecer al mismo grupo)
bronce_erp_clean_df['SITIOWEB'].value_counts(dropna=False)

# EMAIL (13 emails se duplican, aceptable,)
bronce_erp_clean_df['EMAIL'].value_counts(dropna=False)

# TELEFONO (normal)
bronce_erp_clean_df['TELEFONO'].value_counts(dropna=False)

#PRODUCTOS (normal)
bronce_erp_clean_df['PRODUCTOS'].value_counts(dropna=False)

#EMPLEOTOTAL (normal, ojo tipo)
bronce_erp_clean_df['EMPLEOTOTAL'].value_counts(dropna=False)


#RANGOFACTURACION .(habria que realizar correciones, pero probablemente COLUMNA NO NECESARIA)
#bronce_erp_clean_df['RANGOFACTURACION'].value_counts(dropna=False)


# IMPORTA (normal)
bronce_erp_clean_df['IMPORTA'].value_counts(dropna=False)


#EXPORTA (normal)
bronce_erp_clean_df['EXPORTA'].value_counts(dropna=False)




EXPORTA
No    845
si    405
Name: count, dtype: int64